# Metadata curation

Performing metadata curation to standardize terminology, limit unnecessary variance and other needed changes.

### Imports

In [ ]:
import pandas as pd
import awswrangler as wr

### Raw data loading

In [2]:
file_location = "raw_metadata.xlsx"
raw_metadata_table = wr.s3.read_excel(file_location)
metadata_table = raw_metadata_table.copy()

### Clarify column names

In [4]:
metadata_table = metadata_table.rename(
    columns={
        "ASSAY_NAME": "omics",
        "SUB_LOCATION__C": "sub_location",
        "DEIDENTIFIED_MASTER_PATIENT_ID": "patient_id",
    }
)
metadata_table.columns = metadata_table.columns.str.lower()

In [5]:
metadata_table["omics"] = metadata_table["omics"].replace(
    to_replace={
        "RNASeq": "transcriptomics",
        "Genotyping (Global Screening Array)": "genomics_array",
        "Whole Exome Sequencing": "genomics_exome",
        "Proteomic biomarker panels (Olink)": "proteomics",
    }
)

Fill a new batch column by extracting it from the file path.

In [6]:
metadata_table["batch"] = metadata_table["requested_file_path"].str.split(
    "/", expand=True
)[1]
metadata_table.drop(columns=["requested_file_path"], inplace=True)

Initiate empty sample id column to be filled with the omics unique ids.

In [7]:
metadata_table["sample_id"] = ""

## Transcriptomics

In [8]:
# drop R2 files
metadata_table = metadata_table.loc[
    (~metadata_table["raw_data_file_name"].str.contains("_R2"))
].copy()

Get the sample ids from the file name.

In [10]:
transcriptomics = metadata_table.loc[
    (metadata_table["omics"] == "transcriptomics"),
    "raw_data_file_name",
].str.split("_", expand=True)[0]
transcriptomics_idx = transcriptomics.index

metadata_table.loc[transcriptomics_idx, "sample_id"] = transcriptomics

## Genomics (exome)

Get the sample ids from the file name.

In [11]:
genomics_exome = metadata_table.loc[
    metadata_table["omics"].str.contains("genomics_exome"), "raw_data_file_name"
].str.split(".", expand=True)[0]
genomics_exome_idx = genomics_exome.index

metadata_table.loc[genomics_exome_idx, "sample_id"] = genomics_exome

## Genomics (SNP Array)

Get the sample ids from the file name.


In [12]:
genomics_snp = metadata_table.loc[
    metadata_table["omics"].str.contains("genomics_array"), "raw_data_file_name"
].str.split("_[RG][er]", expand=True, regex=True)[0]
genomics_snp_idx = genomics_snp.index

metadata_table.loc[genomics_snp_idx, "sample_id"] = genomics_snp

Identify duplicate samples because of the luminescence method, and removing the red ones.

In [15]:
# drop red files
print(
    f"Number of samples with Red: {metadata_table.raw_data_file_name.str.contains('_Red').sum()}"
)

Number of samples with Red: 2863


In [14]:
print(
    f"Number of samples with Grn: {metadata_table.raw_data_file_name.str.contains('_Grn').sum()}"
)

Number of samples with Grn: 2863


In [16]:
print(f"Metadata table before dropping genomics array Red: {metadata_table.shape}")
metadata_table = metadata_table.loc[
    (~metadata_table["raw_data_file_name"].str.contains("Red"))
]
print(f"Metadata table after dropping genomics array Red: {metadata_table.shape}")

Metadata table before dropping genomics array Red: (14145, 143)
Metadata table after dropping genomics array Red: (11282, 143)


## Proteomics

Get the sample ids from the file name.

In [17]:
proteomics = metadata_table.loc[
    metadata_table["omics"].str.contains("proteomics"), "raw_data_file_name"
].str.split(".", expand=True)[0]
proteomics_idx = proteomics.index

metadata_table.loc[proteomics_idx, "sample_id"] = proteomics

#### Check for missing values

In [18]:
metadata_table.sample_id.isna().sum()

0

Further investigation found duplicates later on in subsequent analyses. So we decided to dig deeper. We found that 6 patients have duplicate sample ids. We decided to drop these patients from the analysis because:
- it is only 6 samples
- in some cases where the difference is `sub_location`, the `sub_location` is not consistent with the other samples from the same patient
- there were a couple of NaN values as well
- for the Normal vs Erosions or Ulcers values, we couldn't pick one

In [19]:
dup_ids = metadata_table.loc[
    metadata_table.sample_id.duplicated(keep=False)
    & ~(metadata_table.omics == "genomics_array")
].sample_id.unique()
for dup_id in dup_ids:
    for column in metadata_table.columns:
        l = len(metadata_table.loc[metadata_table.sample_id == dup_id, column].unique())
        if l > 1:
            print(
                dup_id,
                column,
                l,
                metadata_table.loc[metadata_table.sample_id == dup_id, column].unique(),
            )

FS04287240 sub_location 2 ['Most Proximal Extent' 'Most Proximal']
8051568126 macroscopic_appearance 2 [nan 'Normal']
8051391228 macroscopic_appearance 2 ['Normal' nan]
8051387535 macroscopic_appearance 2 ['Erosions or Ulcers' 'Normal']
8051568148 macroscopic_appearance 2 ['Normal' 'Erosions or Ulcers']
8051600355 macroscopic_appearance 2 ['Normal' 'Erosions or Ulcers']


In [20]:
# shows that most proximal / extent is only present for this one sample_id... so we can drop it
metadata_table.loc[
    (metadata_table.sub_location == "Most Proximal")
    | (metadata_table["sub_location"] == "Most Proximal Extent")
]

,patient_id,investigation_title,ibd_plexus_data_release_date,investigation_disease,investigation_description,investigation_cont_last_name,investigation_cont_first_name,investigation_cont_email,investigation_cont_inst/org,study_title,...,rectum,sigmoid_colon,right_colon,descending_colon,transverse_colon,endoscopy_60,ostomy,disease_activity_60,batch,sample_id
11656,17485622,Biopsy RNASeq Data Generation,20-OCT-2021,IBD,Generation of RNASeq data from tissue biopsy b...,Hurtado-Lorenzo,Andres,ahurtadolorenzo@crohnscolitisfoundation.org,Crohn's and Colitis Foundation,Biopsy RNASeq Data Generation,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Remission,sparc-genewiz-2021,FS04287240
11657,17485622,Biopsy RNASeq Data Generation,20-OCT-2021,IBD,Generation of RNASeq data from tissue biopsy b...,Hurtado-Lorenzo,Andres,ahurtadolorenzo@crohnscolitisfoundation.org,Crohn's and Colitis Foundation,Biopsy RNASeq Data Generation,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Remission,sparc-genewiz-2021,FS04287240


In [24]:
print(f"Duplicate sample ids: {dup_ids}")

Duplicate sample ids: ['FS04287240' '8051568126' '8051391228' '8051387535' '8051568148'
 '8051600355']


In [25]:
# so dropping thses 6 samples
metadata_table = metadata_table.loc[~metadata_table.sample_id.isin(dup_ids)]

## Simple Tissue

Aggregating the tissues into larger categories.

In [26]:
# characteristics_bio_material --> simple_tissue
tissues_mapping = {
    "Descending Colon": "colon",
    "Transverse Colon": "colon",
    "At 20 cm": "colon",
    "Rectum": "colon",
    "Ascending Colon": "colon",
    "Sigmoid Colon": "colon",
    "Ileum": "small_intestine",
    "Cecum": "small_intestine",
}
metadata_table["simple_tissue"] = (
    metadata_table["characteristics_bio_material"].replace(tissues_mapping).str.lower()
)
metadata_table["characteristics_bio_material"] = (
    metadata_table["characteristics_bio_material"].str.replace(" ", "_").str.lower()
)

In [27]:
metadata_table.simple_tissue.value_counts()

simple_tissue
blood              5723
plasma             2250
colon              1795
small_intestine    1330
other               171
Name: count, dtype: int64

### Simplify Macroscopic Appearance 

In [28]:
inflamed_map = {
    "Normal": "normal",
    "Erosions or Ulcers": "inflamed",
    "Possible inflammation": "poss_inflamed",
}
metadata_table["macroscopic_appearance"] = (
    metadata_table["macroscopic_appearance"].replace(inflamed_map).str.lower()
)

In [29]:
metadata_table.macroscopic_appearance.value_counts()

macroscopic_appearance
normal           2200
inflamed          648
poss_inflamed     281
Name: count, dtype: int64

## UC vs CD

In [30]:
uc_cd_map = {
    "Ulcerative Colitis": "uc",
    "Crohn's Disease": "cd",
    "IBD Unclassified": "ibd_unclassified",
}
metadata_table["diagnosis"] = metadata_table["diagnosis"].replace(uc_cd_map).str.lower()

In [31]:
metadata_table.diagnosis.value_counts()

diagnosis
cd                  7286
uc                  3772
ibd_unclassified     212
Name: count, dtype: int64

In [32]:
metadata_table[
    [
        "sex",
        "diagnosis",
        "endo_category",
        "sub_location",
        "macroscopic_appearance",
        "disease_activity_60",
        "perianal",
    ]
] = metadata_table[
    [
        "sex",
        "diagnosis",
        "endo_category",
        "sub_location",
        "macroscopic_appearance",
        "disease_activity_60",
        "perianal",
    ]
].apply(lambda x: x.str.lower())
metadata_table.drop(columns=["raw_data_file_name"], inplace=True)

In [33]:
metadata_table.head(10)

,patient_id,investigation_title,ibd_plexus_data_release_date,investigation_disease,investigation_description,investigation_cont_last_name,investigation_cont_first_name,investigation_cont_email,investigation_cont_inst/org,study_title,...,sigmoid_colon,right_colon,descending_colon,transverse_colon,endoscopy_60,ostomy,disease_activity_60,batch,sample_id,simple_tissue
0,9014430,Blood DNA Genotyping Data Generation,30-APR-2020,IBD,Generation of global screening array and whole...,Hurtado-Lorenzo,Andres,ahurtadolorenzo@crohnscolitisfoundation.org,Crohn's and Colitis Foundation,Blood DNA Genotyping Data Generation,...,NaN,NaN,NaN,NaN,NaN,No Previous Ostomy,NaN,sparc-genotyping,1203012005,blood
2,9014430,Blood DNA Genotyping Data Generation,30-APR-2020,IBD,Generation of global screening array and whole...,Hurtado-Lorenzo,Andres,ahurtadolorenzo@crohnscolitisfoundation.org,Crohn's and Colitis Foundation,Blood DNA Genotyping Data Generation,...,NaN,NaN,NaN,NaN,NaN,No Previous Ostomy,NaN,sparc-genotyping,203990560025_R01C01,blood
3,9014431,Blood DNA Genotyping Data Generation,30-APR-2020,IBD,Generation of global screening array and whole...,Hurtado-Lorenzo,Andres,ahurtadolorenzo@crohnscolitisfoundation.org,Crohn's and Colitis Foundation,Blood DNA Genotyping Data Generation,...,NaN,NaN,NaN,NaN,NaN,NaN,moderate,sparc-genotyping,203990550092_R09C02,blood
5,9014431,Blood DNA Genotyping Data Generation,30-APR-2020,IBD,Generation of global screening array and whole...,Hurtado-Lorenzo,Andres,ahurtadolorenzo@crohnscolitisfoundation.org,Crohn's and Colitis Foundation,Blood DNA Genotyping Data Generation,...,NaN,NaN,NaN,NaN,NaN,NaN,moderate,sparc-genotyping,1203012008,blood
6,9014432,Blood DNA Genotyping Data Generation,30-APR-2020,IBD,Generation of global screening array and whole...,Hurtado-Lorenzo,Andres,ahurtadolorenzo@crohnscolitisfoundation.org,Crohn's and Colitis Foundation,Blood DNA Genotyping Data Generation,...,NaN,NaN,NaN,NaN,NaN,NaN,remission,sparc-genotyping,203990560001_R09C02,blood
7,9014432,Blood DNA Genotyping Data Generation,30-APR-2020,IBD,Generation of global screening array and whole...,Hurtado-Lorenzo,Andres,ahurtadolorenzo@crohnscolitisfoundation.org,Crohn's and Colitis Foundation,Blood DNA Genotyping Data Generation,...,NaN,NaN,NaN,NaN,NaN,NaN,remission,sparc-genotyping,1203012009,blood
10,9014433,Blood DNA Genotyping Data Generation,30-APR-2020,IBD,Generation of global screening array and whole...,Hurtado-Lorenzo,Andres,ahurtadolorenzo@crohnscolitisfoundation.org,Crohn's and Colitis Foundation,Blood DNA Genotyping Data Generation,...,NaN,NaN,NaN,NaN,NaN,NaN,remission,sparc-genotyping,203990560048_R09C02,blood
11,9014433,Blood DNA Genotyping Data Generation,30-APR-2020,IBD,Generation of global screening array and whole...,Hurtado-Lorenzo,Andres,ahurtadolorenzo@crohnscolitisfoundation.org,Crohn's and Colitis Foundation,Blood DNA Genotyping Data Generation,...,NaN,NaN,NaN,NaN,NaN,NaN,remission,sparc-genotyping,1203012010,blood
12,9014434,Blood DNA Genotyping Data Generation,30-APR-2020,IBD,Generation of global screening array and whole...,Hurtado-Lorenzo,Andres,ahurtadolorenzo@crohnscolitisfoundation.org,Crohn's and Colitis Foundation,Blood DNA Genotyping Data Generation,...,NaN,NaN,NaN,NaN,NaN,NaN,remission,sparc-genotyping,204127370065_R10C01,blood
13,9014434,Blood DNA Genotyping Data Generation,30-APR-2020,IBD,Generation of global screening array and whole...,Hurtado-Lorenzo,Andres,ahurtadolorenzo@crohnscolitisfoundation.org,Crohn's and Colitis Foundation,Blood DNA Genotyping Data Generation,...,NaN,NaN,NaN,NaN,NaN,NaN,remission,sparc-genotyping,1203012012,blood


In [34]:
metadata_table.sample_id.value_counts()

sample_id
1203012005             1
FR20592753             1
FS04286885             1
FS04286887             1
FR20588078             1
                      ..
1203012919             1
8051395421             1
8051395446             1
203990560031_R05C01    1
205097370093_R07C01    1
Name: count, Length: 11270, dtype: int64

Write the output table.

In [33]:
metadata_table.to_parquet("processed_metadata.parquet")